# Reddit AITA Finetuned Model Testing

In [ ]:
%pip install evaluate rouge_score peft transformers datasets accelerate bitsnbytes

In [1]:
import textwrap
from datasets import load_dataset
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoTokenizer
import torch
from random import randrange


In [2]:
from huggingface_hub import login
login()

In [3]:
from datasets import load_dataset
dataset = load_dataset("MattBoraske/reddit-AITA-submissions-and-comments-top-50k")

# Flan-T5 Finetuned Model

In [4]:
FLAN_T5_ENCODER_CONTEXT_WINDOW_SIZE = 1024
FLAN_T5_DECODER_CONTEXT_WINDOW_SIZE = 256

In [5]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xxl")
config = PeftConfig.from_pretrained("MattBoraske/flan-t5-xxl-reddit-AITA-top-50k")
model = AutoModelForSeq2SeqLM.from_pretrained(
  "google/flan-t5-xxl",
  device_map="auto",
  torch_dtype=torch.bfloat16
)
model = PeftModel.from_pretrained(model, "MattBoraske/flan-t5-xxl-reddit-AITA-top-50k")

config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/9.60G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/6.06G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/302M [00:00<?, ?B/s]

In [ ]:
sample = dataset['test'][randrange(len(dataset["test"]))]
sample

In [26]:
input_ids = tokenizer(sample['flanT5_instruction'], max_length=FLAN_T5_ENCODER_CONTEXT_WINDOW_SIZE, padding='max_length', return_tensors="pt", truncation=True).input_ids.cuda()

outputs = model.generate(
  input_ids=input_ids,
  max_new_tokens=FLAN_T5_DECODER_CONTEXT_WINDOW_SIZE,
  repetition_penalty=1.5
)

prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens=True)
print(textwrap.fill(prediction, width=100))

yta. you're not going to buy him the black costume? that's ridiculous.


# Llama-2 Finetuned Model

In [1]:
tokenizer = AutoTokenizer.from_pretrained("MattBoraske/llama-2-13b-chat-reddit-AITA-top-50k")
model = AutoModelForCausalLM.from_pretrained(
  "MattBoraske/llama-2-13b-chat-reddit-AITA-top-50k",
  device_map='auto',
  torch_dtype=torch.bfloat16
)

tokenizer_config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

In [ ]:
sample = dataset['test'][randrange(len(dataset["test"]))]
sample

In [ ]:
input_ids = tokenizer(sample['llama2_instruction'], return_tensors="pt").input_ids.cuda()

outputs = model.generate(
  input_ids=input_ids,
  max_new_tokens=256,
  repetition_penalty=1.1
)

prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens=True)
print(textwrap.fill(prediction, width=100))